In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split

from category_encoders import TargetEncoder

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score



from sklearn.preprocessing import OneHotEncoder

from scipy.sparse import hstack, vstack

from sklearn.neural_network import MLPRegressor





# Data load:

In [ ]:
df_dev = pd.read_csv('dev.tsv', sep='\t')
df_dev.drop_duplicates(inplace = True)
df_dev=df_dev[df_dev['quality']>=0]

df_eval = pd.read_csv('eval.tsv', sep='\t')

df = pd.concat([df_dev, df_eval], sort=False, ignore_index=True)
len(df_dev)

In [ ]:
# missing_values_dev = len(df_dev)-len(df_dev.drop_duplicates()), 
# missing_values_eval = len(df_eval)-len(df_eval.drop_duplicates()), 
# missing_values_df = len(df)-len(df.drop_duplicates())

# missing_values_dev, missing_values_eval, missing_values_df

In [ ]:
# 38311-35716

# Plot

In [ ]:
y = df_dev.drop('quality', axis=1).columns
x = df_dev.drop('quality', axis=1).isnull().sum()

fig, ax = plt.subplots()
sns.set(rc={'figure.figsize':(6,4)})
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5) 

ax.set(xlabel='Number of missing values')
ax.set(ylabel='Features')
ax.set_xlim(0,85000)
ax.set_xticks(np.arange(0,86000,15000))

ax = sns.barplot(x=x, y=y, color="cornflowerblue", saturation=1)#.set_title('Null values')
#fig.savefig("missingValues2.pdf",bbox_inches='tight')
x

In [ ]:
ax = sns.displot(df_dev['quality'], kde=True)
mean = df_dev['quality'].mean()
plt.axvline(mean, 0,0.41, color='r', linestyle='--')

# #https://stackoverflow.com/questions/51417483/mean-median-mode-lines-showing-only-in-last-graph-in-seaborn

In [ ]:
f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw= {"height_ratios": (0.2, 1)})
mean=df_dev["quality"].mean()
#median=df_dev["quality"].median()
#mode=df_dev["quality"].mode().get_values()[0]

sns.boxplot(df_dev["quality"], ax=ax_box, color='cornflowerblue', saturation=1)
ax_box.axvline(mean, color='r', linestyle='--', label='mean')
#ax_box.axvline(median, color='g', linestyle='-')
#ax_box.axvline(mode, color='b', linestyle='-')

sns.distplot(df["quality"], ax=ax_hist,  color='cornflowerblue')
ax_hist.axvline(mean, color='r', linestyle='--', label='mean')
# ax_hist.axvline(median, color='g', linestyle='-')
# ax_hist.axvline(mode, color='b', linestyle='-')

plt.legend()
ax_box.set(xlabel='')
#ax.set_xticks(np.arange(0,130000,20000))


plt.show()

#f.savefig("qualityDensity.pdf",bbox_inches='tight')


# Drop duplicates

In [ ]:
(df_dev['description'].value_counts().values>1).sum()

In [ ]:
not_unique_description = df_dev['description'].value_counts()[df_dev['description'].value_counts().values > 1].index

df_description_mask=df_dev['description'].isin(not_unique_description)
df_description = df_dev[df_description_mask]
#df_description

In [ ]:
mask_duplicate_descr = [not elem for elem in df_description_mask]
df_dev = df_dev[mask_duplicate_descr]
#df_dev

# Description_length:

In [ ]:
# df_dev["Description_Length"] = [len(desc.split()) for desc in df_dev['description']]
# df_eval["Description_Length"] = [len(desc.split()) for desc in df_eval['description']]
# df["Description_Length"] = [len(desc.split()) for desc in df['description']]

# df_dev

In [ ]:
# import numpy as np
# import seaborn as sns
# sns.set_theme(style="ticks")


# sns.set_context("paper", font_scale=1.5) 

# ax = sns.jointplot(x=df_dev['quality'], y=df_dev['Description_Length'], kind="hex", color="#4CB391")
# ax.set_ylim(0,100)
# ax.savefig("description_length.pdf",bbox_inches='tight')


In [ ]:
# my_rho = np.corrcoef(df_dev['quality'], df_dev['Description_Length'])

# print(my_rho)

# Column cleaning:

In [ ]:
columns = df.columns[0:8] #non prendo la colonna quality e descr_length

for column in columns:
    df[column] = df[column].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    
for column in columns:
    df_dev[column] = df_dev[column].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    
for column in columns:
    df_eval[column] = df_eval[column].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Encoding:

In [ ]:
X_dev = df_dev.drop(labels = ['description','quality'], axis=1)#.values
y_dev = df_dev['quality']#.values[:, np.newaxis]

In [ ]:
X_eval = df_eval.drop(labels = ['description'], axis=1)#.values
X_dev

In [ ]:
# df_dev1 = df_dev.drop(labels = ['description','region_2','country'], axis=1)
# df_dev1.drop_duplicates(inplace = True)
# df_dev1
# X_dev = df_dev1.drop(labels = ['quality'], axis=1)
# y_dev = df_dev1['quality']
# X_eval = df_eval.drop(labels = ['description','region_2', 'country'], axis=1)#.values


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import VotingRegressor

from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


In [ ]:
# designation = df_dev['designation'].value_counts()

# N_entries = 10  #we encode when winery category is at least 2 times frequent
# mask = designation.values >= N_entries
# top_frequent_designation=designation[mask].index

# df_designation_mask=df_dev['designation'].isin(top_frequent_designation)
# df['tf_winery']=df_dev['winery'][df_designation_mask]
# df['tf_winery'].fillna(value='other',inplace=True) 


In [ ]:
X_train_to_encode, X_test_to_encode, y_train, y_test = train_test_split(X_dev, y_dev, train_size= 0.8, shuffle=True, random_state=42)

In [ ]:
encoder = TargetEncoder(cols=['designation','region_1','province','variety','winery', 'country','region_2'])

In [ ]:
X_train_encoded = encoder.fit_transform(X_train_to_encode, y_train)
X_test_encoded = encoder.transform(X_test_to_encode)

X_dev_encoded = encoder.fit_transform(X_dev, y_dev)
X_eval_encoded = encoder.transform(X_eval)

X_train_encoded

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from scipy.sparse import hstack
# X_train_encoded

In [ ]:
#TFIDF_norm.toarray()

In [ ]:
# TFIDF_matrix_test=v.transform(X_test_to_encode['description'])


In [ ]:
# X_test_encoded = hstack((X_test_encoded.values.astype(float), TFIDF_matrix_test))


In [ ]:
# cols_to_norm = [0,1,2,3,4,5,6]
# X_train_encoded[cols_to_norm] = X_train_encoded[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
# X_test_encoded[cols_to_norm] = X_train_encoded[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# X_dev_encoded.describe()

# Machine Learning:

In [ ]:
reg = RandomForestRegressor(n_estimators=128, max_features='sqrt')
reg.fit(X_train_encoded, y_train)
y_hat = reg.predict(X_test_encoded)
y_hat.mean(), y_hat.std()

In [ ]:
r2_score(y_test, y_hat)

In [ ]:
feature_names = X_train_encoded.columns
sorted(zip(feature_names, reg.feature_importances_), key=lambda x: x[1],reverse=True)

In [ ]:
names = ['winery','designation','region_1','variety','province','country','region_2']
importance = [0.660, 0.148, 0.077, 0.0613, 0.028, 0.017, 0.009]

fig, ax = plt.subplots()
sns.set(rc={'figure.figsize':(6,4)})
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5) 

ax.set(xlabel='Feature importance')
ax.set(ylabel='Features')
sns.color_palette("pastel")

sns.barplot(x=importance, y=names)

#fig.savefig("featureImportance.pdf",bbox_inches='tight')


In [ ]:
reg = KNeighborsRegressor(n_neighbors=50, weights='distance', algorithm='auto')
reg.fit(X_train_encoded, y_train)
y_hat = reg.predict(X_test_encoded)
r2_score(y_test, y_hat)

In [ ]:
# from sklearn.svm import LinearSVR
# reg = LinearSVR(max_iter=10000)
# reg.fit(X_train_encoded, y_train)
# y_hat = reg.predict(X_test_encoded)
# y_hat.mean(), y_hat.std()

In [ ]:
# r2_score(y_test,y_hat)

In [ ]:
# reg = MLPRegressor(hidden_layer_sizes=(64,128,128,128,256,256,256,128),
#                    verbose=True, random_state=9, early_stopping=True)
# reg.fit(X_dev_encoded, y_dev)

In [ ]:
reg = make_pipeline(PolynomialFeatures(5), Ridge(alpha=0.5, tol=0.2))
reg.fit(X_train_encoded, y_train)
y_hat = reg.predict(X_test_encoded)
y_hat.mean(), y_hat.std()
r2_score(y_test, y_hat)

In [ ]:
from sklearn.model_selection import ParameterGrid

params = {
    #"max_depth": [None], #None
    #"min_impurity_decrease": [0], #0
    #"criterion": ['mse'] #mse
    "n_estimators": [128,256,512],
    "max_features": ["sqrt"],  #ho levato "auto" che sembra il meno performante, ho levato log2 perchè funziona uguale a sqrt ma ci mette più tempo
    #"max_leaf_nodes" : [9000, 10000, 11000,], #funziona meglio di None
    #"min_samples_split": [2,3], #2
    #"min_samples_leaf": [1,2], #1
    "random_state": [42], # always use the samet random seed
    "n_jobs": [-1], # for parallelization
}

r2s = []
for config in ParameterGrid(params):
    reg = RandomForestRegressor(**config)
    reg.fit(X_train_encoded, y_train)
    r2s.append(r2_score(y_test, reg.predict(X_test_encoded)))
    print(config, r2_score(y_test, reg.predict(X_test_encoded)))


In [ ]:
from sklearn.model_selection import ParameterGrid

params = {
    "n_neighbors" : [25,30,35,40,45,50,55,60], #30
    "weights" : ["uniform", "distance"], #distance
    "p" : [1,2], #1
    "n_jobs" : [-1]
}

r2s = []
for config in ParameterGrid(params):
    reg = KNeighborsRegressor(**config)
    reg.fit(X_train_encoded, y_train)
    r2s.append(r2_score(y_test, reg.predict(X_test_encoded)))
    print(config, r2_score(y_test, reg.predict(X_test_encoded)))

In [ ]:
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.feature_extraction import text
# from sklearn.feature_extraction.text import CountVectorizer

# corpus = df_dev["description"].values
# Y = df_dev["quality"].values

# customStopWords = text.ENGLISH_STOP_WORDS.union(['wine', '2009', '2010','2011', '2012', '2013', '2014', '2015','2016', '2017', '2018',
#                                                  '2019', '2020', '2021', '2022','2023', '2024', '2025', '2030', '100', '10', '12',
#                                                  '14', '15', '20', '25', '30','40', '50', '60', '70', '90'])

# CV = CountVectorizer(stop_words=customStopWords, max_features=20, ngram_range=(1,2))
# X = CV.fit_transform(corpus) #Let's be careful here, X is a sparse Matrix

# print("Number of entries (rows):", X.shape[0],\
#       "\nNumber of features (columns):", X.shape[1])

In [ ]:
# X_array = X.toarray() #Convert X from a sparse matrix to a usual matrix

# inverted_dict = dict([[v,k] for k,v in CV.vocabulary_.items()]) # {X_array column index: "word"}
# final_dict = {} # {"word": total number of instances }

# for x in range(len(X_array[0,:])): #Fill the final dict
#     final_dict[inverted_dict[x]]=np.sum(X_array[:,x]) 

# print("20 most frequent words:",sorted(final_dict.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)[0:20]) #Display of the final dict

In [ ]:
import seaborn
seaborn.scatterplot(y_test.ravel(),y_hat.ravel())

In [ ]:
def target_encoding (X_train_to_encode, y_train, X_test_to_encode, y_test):
    encoder = TargetEncoder()
    X_train_encoded = encoder.fit_transform(X_train_to_encode, y_train)
    X_test_encoded = encoder.transform(X_test_to_encode)
    return X_train_encoded, X_test_encoded

In [ ]:
def generate_train_test(X, y):
    X_train_to_encode, X_test_to_encode, y_train, y_test = train_test_split(X_dev, y_dev, test_size= 0.2, shuffle=True, random_state=42)
    X_train, X_test = target_encoding(X_train_to_encode,y_train,X_test_to_encode,y_test)
    return X_train, X_test, y_train, y_test

In [ ]:
def evaluate_model(X, y, model, model_name):
    X_train, X_test, y_train, y_test = generate_train_test(X, y)
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    return mean_squared_error(y_test, y_hat), r2_score(y_test, y_hat)

In [ ]:
from prettytable import PrettyTable
degree = 5
models = [
    make_pipeline(PolynomialFeatures(degree), Lasso(alpha=0.5, tol=0.2)),
    make_pipeline(PolynomialFeatures(degree), Ridge(alpha=0.5, tol=0.2)),
    LinearRegression(),
    Ridge(random_state=42),
    MLPRegressor(),
    RandomForestRegressor(n_estimators=10),
    KNeighborsRegressor()

]
names = [
    'polyn+lasso',
    'polyn+ridge',
    'linreg',
    'ridge',
    'mlp_standard',
    'rf',
    'KNN',

]


In [ ]:
t = PrettyTable()
t.field_names = ['model', 'MSE', 'R2']
for model, name in zip(models, names):
    mse, r2 = evaluate_model(X_dev, y_dev, model, name)
    t.add_row([name, mse, r2])
    print(t)


In [ ]:
# vectorizer = TfidfVectorizer(stop_words="english", binary=True, use_idf=False, norm=False)
# wpm = vectorizer.fit_transform(df_dev["description"])
# N = 150
# freq = sorted(zip(vectorizer.get_feature_names(), wpm.sum(axis=0).tolist()[0]), key=lambda x: x[1], reverse=True)[:N]

# # mask to be used to filter columns in wpm (only keeps the ones for the 100 most␣frequent words)
# words = [ word for word, _ in freq ]
# mask = [ w in words for w in vectorizer.get_feature_names() ]
# words_ = [ w for w in vectorizer.get_feature_names() if w in words ]
# words_df = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(), columns=[f"word_{word}" for word in words_], index=df_dev.index)

# # Only encode "room_type"
# df_dev = pd.get_dummies(df, columns=['room_type'])
# df_dev = df_dev.join(pd.DataFrame(data=wpm[:, np.array(mask)].toarray(), columns=[f"word_{word}" for word in words_], index=df_dev.index))
# # discard "neighbourhood" and "neighbourhood_group"
# df_dropped = df_1h.drop(columns=["neighbourhood_group","neighbourhood", "name", "host_name", "last_review"])
# df_dropped["id"] = df_dropped.index
# train_valid_mask = ~df_dropped["price"].isna()
# feature_names = df_dropped[train_valid_mask].drop(columns=["price"]).columns

In [ ]:
# columns = df_dev.columns

# for column in columns:
#     category_number = df_dev[column].value_counts()
#     print(column ,':', (category_number<=1).sum(), '/', len(df_dev[column].unique()))

# Submit:

In [ ]:
import csv
reg = RandomForestRegressor(n_estimators=1028, max_leaf_nodes= None, max_features = 'sqrt' )
reg.fit(X_dev_encoded, y_dev)

y_final_pred = reg.predict(X_eval_encoded)
pd.DataFrame(y_final_pred).to_csv("output.csv", index_label="Id", header=["Predicted"])

# Part 2

In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split

from category_encoders import TargetEncoder

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score



from sklearn.preprocessing import OneHotEncoder

from scipy.sparse import hstack, vstack

from sklearn.neural_network import MLPRegressor

In [137]:
df_dev = pd.read_csv('dev.tsv', sep='\t')
#df_dev.drop_duplicates(inplace = True)
df_dev=df_dev[df_dev['quality']>=0]

df_eval = pd.read_csv('eval.tsv', sep='\t')

df = pd.concat([df_dev, df_eval], sort=False, ignore_index=True)
len(df_dev)
df_dev

,country,description,designation,province,region_1,region_2,variety,winery,quality
0,France,"A creamed pear wine, with an attractive tang o...",Brut Blanc de Blancs,Alsace,Crémant d'Alsace,NaN,Pinot Blanc,Lucien Albrecht,45.0
1,US,"Simple and dry, this Cabernet has modest black...",NaN,California,Paso Robles,Central Coast,Cabernet Sauvignon,Castle Rock,31.0
2,US,"This lovely wine captures the floral, perfumed...",NaN,Oregon,Willamette Valley,Willamette Valley,Gewürztraminer,Château Bianca,35.0
3,Portugal,"The aromas are the thing here, as so often wit...",NaN,Alentejano,NaN,NaN,Touriga Nacional,Herdade do Esporão,41.0
4,Italy,"This is an interesting, outright strange wine ...",Natì,Southern Italy,Pompeiano,NaN,Coda di Volpe,Sorrentino,37.0
...,...,...,...,...,...,...,...,...,...
120730,France,"Moët's style, with its delicious forward fruit...",Brut,Champagne,Champagne,NaN,Champagne Blend,Moët & Chandon,59.0
120732,Portugal,"This is soft, young and fruity, with a dominat...",PV Ruby Port Collections,Port,NaN,NaN,Port,Barão de Vilar,42.0
120733,US,"Showing ripe peach, pineapple and honeysuckle ...",NaN,California,Yountville,Napa,Chardonnay,Liparita,46.0
120735,US,A first release from this new Walla Walla wine...,NaN,Washington,Walla Walla Valley (WA),Columbia Valley,Syrah,Delmas,58.0


In [138]:
columns = df.columns[0:8] #non prendo la colonna quality e descr_length

for column in columns:
    df[column] = df[column].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    
for column in columns:
    df_dev[column] = df_dev[column].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    
for column in columns:
    df_eval[column] = df_eval[column].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [139]:
not_unique_description = df_dev['description'].value_counts()[df_dev['description'].value_counts().values > 1].index

df_description_mask=df_dev['description'].isin(not_unique_description)
df_description = df_dev[df_description_mask]
#df_description
mask_duplicate_descr = [not elem for elem in df_description_mask]
df_dev = df_dev[mask_duplicate_descr]
#df_dev

# Tf-idf description:

In [140]:
from sklearn.preprocessing import MaxAbsScaler

v = TfidfVectorizer(stop_words='english',ngram_range=(1, 2), max_features = 2000)
TFIDF_matrix = v.fit_transform(df_dev['description'])

scaler = MaxAbsScaler()
TFIDF_norm = scaler.fit_transform(TFIDF_matrix)
TFIDF_norm

<85028x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 1889731 stored elements in Compressed Sparse Row format>

# One-hot Encoding

#### Winery

In [141]:
winery = df_dev['winery'].value_counts()

N_entries = 2  #we encode when winery category is at least 2 times frequent
mask = winery.values >= N_entries
top_frequent_winery=winery[mask].index

df_winery_mask=df_dev['winery'].isin(top_frequent_winery)
df_dev['tf_winery']=df_dev['winery'][df_winery_mask]
df_dev['tf_winery'].fillna(value='other',inplace=True) 

enc1 = OneHotEncoder(handle_unknown='ignore')
df_1h_winery=enc1.fit_transform(df_dev['tf_winery'].values.reshape(-1, 1)) #with reshape we have a column vector
df_1h_winery

<85028x9663 sparse matrix of type '<class 'numpy.float64'>'
	with 85028 stored elements in Compressed Sparse Row format>

#### Designation

In [142]:
designation = df_dev['designation'].value_counts()

N_entries = 2  #we encode when winery category is at least 2 times frequent
mask = designation.values >= N_entries
top_frequent_designation=designation[mask].index

df_designation_mask=df_dev['designation'].isin(top_frequent_designation)
df_dev['tf_designation']=df_dev['designation'][df_designation_mask]
df_dev['tf_designation'].fillna(value='other',inplace=True) 

enc2 = OneHotEncoder(handle_unknown='ignore')
df_1h_designation=enc2.fit_transform(df_dev['tf_designation'].values.reshape(-1, 1)) #with reshape we have a column vector

In [143]:
df_1h_designation

<85028x8876 sparse matrix of type '<class 'numpy.float64'>'
	with 85028 stored elements in Compressed Sparse Row format>

#### Region_1

In [144]:
region_1 = df_dev['region_1'].value_counts()

N_entries = 10  #we encode when winery category is at least 2 times frequent
mask = region_1.values >= N_entries
top_frequent_region_1=region_1[mask].index


df_region_1_mask=df_dev['region_1'].isin(top_frequent_region_1)
df_dev['tf_region_1']=df_dev['region_1'][df_region_1_mask]
df_dev['tf_region_1'].fillna(value='other',inplace=True) 

enc3 = OneHotEncoder(handle_unknown='ignore')
df_1h_region_1=enc3.fit_transform(df_dev['tf_region_1'].values.reshape(-1, 1)) #with reshape we have a column vector
df_1h_region_1

<85028x552 sparse matrix of type '<class 'numpy.float64'>'
	with 85028 stored elements in Compressed Sparse Row format>

#### Variety

In [145]:
# variety = df_dev['variety'].value_counts()

In [146]:
# N_entries = 5  #we encode when winery category is at least 2 times frequent
# mask = variety.values >= N_entries
# top_frequent_variety=variety[mask].index

In [147]:
# df_variety_mask=df_dev['variety'].isin(top_frequent_variety)
# df_dev['tf_variety']=df_dev['variety'][df_variety_mask]
# df_dev['tf_variety'].fillna(value='other',inplace=True) 

In [148]:
# enc = OneHotEncoder(handle_unknown='ignore')
# df_1h_variety=enc.fit_transform(df_dev['tf_variety'].values.reshape(-1, 1)) #with reshape we have a column vector
# df_1h_variety

#### General encoding

In [149]:
df_quality = df_dev['quality']

df = pd.concat([df_dev, df_eval], sort=False, ignore_index=True)

df.drop(labels=['country','winery','description','designation','quality','region_2','region_1', 'tf_winery','tf_designation','tf_region_1'],axis=1,inplace=True)
df

,province,variety
0,alsace,pinot blanc
1,california,cabernet sauvignon
2,oregon,gewurztraminer
3,alentejano,touriga nacional
4,southern italy,coda di volpe
...,...,...
115209,california,bordeaux-style red blend
115210,california,cabernet franc
115211,california,g-s-m
115212,polkadraai hills,chenin blanc-sauvignon blanc


In [150]:
dev_rows = df_dev.shape[0]

df_1h_total = pd.get_dummies(df,sparse=True)   #one hot encoding of the categorical attribute
df_1h_total

df_1h = df_1h_total.values[:dev_rows, :]
df_1h_eval =df_1h_total.values[dev_rows:, :]

In [151]:
df_1h = hstack([df_1h, df_1h_winery, df_1h_designation, df_1h_region_1, TFIDF_norm])
df_1h = df_1h.tocsr()  #convert this matrix to Compressed Sparse Row format
df_1h

<85028x22178 sparse matrix of type '<class 'numpy.float64'>'
	with 2314868 stored elements in Compressed Sparse Row format>

In [152]:
# X_dev=df_1h[:dev_rows,:]
# y_dev=df_quality[:dev_rows]

# X_eval=df_1h[dev_rows:,:]


# Hold out validation

In [153]:
#X_train, X_valid, y_train, y_valid = train_test_split(df_1h,df_quality, test_size= 0.2, shuffle=True, random_state=42)

In [154]:
#X_train.shape

# Regression Models

In [155]:
reg = MLPRegressor(activation='tanh', 
                   #hidden_layer_sizes=(256, 256),
                   random_state=42,
                   validation_fraction=0.1,
                                verbose=True,
                                early_stopping=True)
reg.fit(df_1h, df_quality)
# y_pred = reg.predict(X_valid) 
 
# r2 = r2_score(y_valid, y_pred)
# print(r2)

Iteration 1, loss = 98.02250689
Validation score: 0.653423
Iteration 2, loss = 19.07863574
Validation score: 0.709549
Iteration 3, loss = 13.82015367
Validation score: 0.729274
Iteration 4, loss = 10.38585379
Validation score: 0.732984
Iteration 5, loss = 7.90194386
Validation score: 0.731270
Iteration 6, loss = 5.97339059
Validation score: 0.728530
Iteration 7, loss = 4.48197320
Validation score: 0.721087
Iteration 8, loss = 3.36111867
Validation score: 0.710010
Iteration 9, loss = 2.62460121
Validation score: 0.706913
Iteration 10, loss = 2.12615042
Validation score: 0.709532
Iteration 11, loss = 1.80857846
Validation score: 0.703490
Iteration 12, loss = 1.56125003
Validation score: 0.699985
Iteration 13, loss = 1.41603043
Validation score: 0.695938
Iteration 14, loss = 1.32146562
Validation score: 0.698453


C:\Users\danie\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:587: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPRegressor(early_stopping=True, hidden_layer_sizes=(256, 256),
             random_state=42, verbose=True)

In [156]:
# y_pred = reg.predict(X_valid)
# r2 = r2_score(y_valid, y_pred)
# print(r2)

# Submit

In [157]:
TFIDF_matrix_eval = v.transform(df_eval['description'])

TFIDF_norm_eval = scaler.transform(TFIDF_matrix_eval)
TFIDF_norm_eval

<30186x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 670620 stored elements in Compressed Sparse Row format>

In [158]:
df_winery_mask_eval=df_eval['winery'].isin(top_frequent_winery)
df_eval['tf_winery']=df_eval['winery'][df_winery_mask_eval]
df_eval['tf_winery'].fillna(value='other',inplace=True) 

df_1h_winery_eval=enc1.transform(df_eval['tf_winery'].values.reshape(-1, 1)) #with reshape we have a column vector
df_1h_winery_eval

<30186x9663 sparse matrix of type '<class 'numpy.float64'>'
	with 30186 stored elements in Compressed Sparse Row format>

In [159]:
df_designation_mask_eval=df_eval['designation'].isin(top_frequent_winery)
df_eval['tf_designation']=df_eval['designation'][df_winery_mask_eval]
df_eval['tf_designation'].fillna(value='other',inplace=True) 

df_1h_designation_eval=enc2.transform(df_eval['tf_designation'].values.reshape(-1, 1)) #with reshape we have a column vector
df_1h_designation_eval

<30186x8876 sparse matrix of type '<class 'numpy.float64'>'
	with 23063 stored elements in Compressed Sparse Row format>

In [160]:
df_region_1_mask_eval=df_eval['region_1'].isin(top_frequent_region_1)
df_eval['tf_region_1']=df_eval['region_1'][df_region_1_mask_eval]
df_eval['tf_region_1'].fillna(value='other',inplace=True) 

df_1h_region_1_eval=enc3.transform(df_eval['tf_region_1'].values.reshape(-1, 1)) #with reshape we have a column vector
df_1h_region_1_eval

<30186x552 sparse matrix of type '<class 'numpy.float64'>'
	with 30186 stored elements in Compressed Sparse Row format>

In [161]:
df_1h_eval.shape

(30186, 1087)

In [162]:
df_1h_eval = hstack([df_1h_eval, df_1h_winery_eval, df_1h_designation_eval, df_1h_region_1_eval, TFIDF_norm_eval])
df_1h_eval = df_1h_eval.tocsr()
df_1h_eval

<30186x22178 sparse matrix of type '<class 'numpy.float64'>'
	with 814427 stored elements in Compressed Sparse Row format>

In [163]:
import csv
y_final_pred = reg.predict(df_1h_eval)
pd.DataFrame(y_final_pred).to_csv("output.csv", index_label="Id", header=["Predicted"])

# Linear SVR

In [178]:
from sklearn.svm import LinearSVR
reg = LinearSVR()
param_grid = {'epsilon':[2], 'C':[7,8,9], 'fit_intercept':[True], 'max_iter' : [5000,6000]} #,'max_iter':[1000,2000,3000,4000,5000,6000]
gridsearch = GridSearchCV(reg, param_grid, scoring='r2', cv=5)

gridsearch.fit(df_1h, df_quality)
print(gridsearch.best_params_)

{'C': 9, 'epsilon': 2, 'fit_intercept': True, 'max_iter': 6000}


In [179]:
from sklearn.model_selection import GridSearchCV
reg = LinearSVR(epsilon=2, C=7, max_iter=5000)
scores = cross_val_score(reg, df_1h, df_quality, cv=5, scoring='r2')
print(scores.mean())

0.737711425960917


In [181]:
import csv
reg.fit(df_1h,df_quality)
y_final_pred = reg.predict(df_1h_eval)
pd.DataFrame(y_final_pred).to_csv("output.csv", index_label="Id", header=["Predicted"])